In [138]:
import pandas as pd
import numpy as np

def epd(factor):
    factor2=factor.copy()
    factor2.iloc[:,0]=0
    factor2['prob']=1-factor['prob']
    frames=[factor,factor2]
    result=pd.concat(frames)
    return result


E=np.array([1,0.0003]).reshape(1,2)
E=pd.DataFrame(data=E,columns=['E','prob'])
E=epd(E)


B=np.array([1,0.0001]).reshape(1,2)
B=pd.DataFrame(data=B,columns=['B','prob'])
B=epd(B)

WA=np.array([1,0,0.4,
            1,1,0.8]).reshape(2,3)
WA=pd.DataFrame(data=WA,columns=['W','A','prob'])
WA=epd(WA)


GA=np.array([1,0,0.04,
            1,1,0.4]).reshape(2,3)
GA=pd.DataFrame(data=GA,columns=['G','A','prob'])
GA=epd(GA)


ABE=np.array([1,0,0,0.01,
            1,0,1,0.2,
            1,1,0,0.95,
            1,1,1,0.96]).reshape(4,4)
ABE=pd.DataFrame(data=ABE,columns=['A','B','E','prob'])
ABE=epd(ABE)




# Restrict

In [139]:
def restrict(factor,variable,value):
    return factor.loc[factor[variable] == value].loc[:, factor.columns != variable]

# Sumout

In [140]:
def sumout(factor,variable):
    print("Sumout",variable, "from:")
    print(factor)
    variables_list = list(factor.columns.values)
    variables_list.remove(variable)
    variables_list.remove("prob")
    print("Result: ")
    result = factor.groupby(variables_list).sum().reset_index()
    print(result.drop([variable],axis=1))
    print("---------------")
    return result.drop([variable],axis=1)

# Multiply

In [141]:
def product_factor(factor1, factor2):
    print("Multiplying:")
    print(factor1)
    print(factor2)
    def repeat(df,var):
        copy1 = df.copy()
        copy1[var] = 1
        copy2 = df.copy()
        copy2[var] = 0
        return pd.concat([copy1, copy2])
    df1 = factor1.copy()
    df2 = factor2.copy()
    #rename column prob
    df1["prob1"] = df1["prob"]
    df1=df1.drop(["prob"], axis=1)
    df2["prob2"] = df2["prob"]
    df2 = df2.drop(["prob"], axis=1)
    

    common_columns = set(df1.columns).intersection(set(df2.columns))
    aa = []
    for item in common_columns:
        aa.append(item)
    
    #if common columns exist
    if aa != []:
        result = df1.merge(df2)
        result["prob"] = result["prob1"]*result["prob2"]
        print("Result:")
        result = result.drop(['prob1', 'prob2'], axis=1)
        print(result)
        print("---------------")
        return result
    
    #if common columns do not exist
    elif aa == []:
        df1['tmp'] = 1
        df2['tmp'] = 1

        df = pd.merge(df1, df2, on=['tmp'])
        df = df.drop('tmp', axis=1)
        df["prob"] = df["prob1"] *df["prob2"] 
        df = df.drop("prob1", axis=1)
        df = df.drop("prob2", axis=1)
        print("Result:")
        print(df)
        print("---------------")
        return df
    
        

# Normalize

In [142]:
def normalize(factor):
    print("Normalize:")
    print(factor)
    df = factor.copy()
    total = sum(df["prob"])
    df["prob"] = round(df["prob"]/total,8)
    print("Result:")
    print(df)
    print("---------------")
    return df

In [143]:
def inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars): #"AB", "AS"
    #iterlate all hidden variables
    for var in ordered_hidden_var_list:
        cal_list =[]
        new_factor_list =[]
        #multiply all the factors involve the hidden variables then sumout
        for factor in factor_list:
            if var in list(factor.columns):
                cal_list.append(factor) 
            else:
                new_factor_list.append(factor)
        while len(cal_list) > 1:
            cal_list = [product_factor(cal_list[0],cal_list[1])] + cal_list[2:]
        new_factor_list = [sumout(cal_list[0],var)]+new_factor_list    
        factor_list = new_factor_list      
    while len(factor_list) >1:
        factor_list = [product_factor(factor_list[0],factor_list[1])] + factor_list[2:]
    result = factor_list[0]
    if evidence_vars != []:
        for item in evidence_vars:
            result = restrict(result,item[0],item[1])
        result = normalize(result)
    elif evidence_vars == []:
        result = normalize(factor_list[0])


In [144]:
factor_list=[]
factor_list.append(E)
factor_list.append(B)
factor_list.append(WA)
factor_list.append(ABE)
factor_list.append(GA)
ordered_hidden_var_list = ["A","B","E","G"]
evidence_vars =[]
query_variables=["W"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [154]:
factor_list=[]
factor_list.append(E)
factor_list.append(B)
factor_list.append(WA)
factor_list.append(ABE)
factor_list.append(GA)
ordered_hidden_var_list = ["A","B","E","W"]
evidence_vars =[]
query_variables=["G"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [ ]:
#p(w) = 0.40406

In [ ]:
#part2 check p(G=1|W =1) and p(G = 1)

In [145]:
ordered_hidden_var_list = ["A","B","E"]
evidence_vars =[["W",1.0]]
query_variables=["G"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [146]:
ordered_hidden_var_list = ["A","B","E"]
evidence_vars =[["W",0]]
query_variables=["G"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [153]:
ordered_hidden_var_list = ["B","E"]
evidence_vars =[["W",1],["G",0]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

# p(G=1|W =1) =  0.408039

In [42]:
ordered_hidden_var_list = ["A","B","E","W"]
evidence_vars =[]
query_variables=["G"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [ ]:
#p(G=1|W =0)  = 0.401363

In [ ]:
#p(G=1|W =1)  = 0.408039

In [ ]:
#3A  P(A) ?=P(A|W)

In [43]:
ordered_hidden_var_list = ["B","E","G","W"]
evidence_vars =[]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [44]:
#p(A=1) = 0.010151

In [46]:
ordered_hidden_var_list = ["B","E","G"]
evidence_vars =[["W",1]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [ ]:
#P(A=1|W=1) = 0.020098

In [165]:
ordered_hidden_var_list = ["B","E","G"]
evidence_vars =[["W",0]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [ ]:
#P(A=1|W=1) = 0.020098

In [ ]:
#3B
#p(A = 1|W=1) ?= p(A = 1|W=1, G = 1) 

#P(A=1|W=1) = 0.020098
#P(A=1|G=1) = 0.020098
#p(A = 1|W=1, G = 1) =0.039404

In [49]:
ordered_hidden_var_list = ["B","E"]
evidence_vars =[["G",1],["W",1]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [ ]:
#p(A = 1|W=1, G = 1) =0.039404

In [ ]:
#3C

#P(A=1|G=1) ? = P(A=1|W=1,G=1)

#P(A=1|G=1) = 0.020098
#p(A=1|W=1, G=1) =0.039404

In [51]:
ordered_hidden_var_list = ["B","E","W"]
evidence_vars =[["G",1]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [52]:
#p(A=1|W=0, G=0) 
ordered_hidden_var_list = ["B","E"]
evidence_vars =[["G",0],["W",0]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [ ]:
#p(A=1|W=0, G=0) = 0.001138

In [53]:
#p(A=1|W=0, G=0) 
ordered_hidden_var_list = ["B","E"]
evidence_vars =[["G",1],["W",0]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [ ]:
#p(A=1|W=0, G=1) = 0.00679

In [166]:
#p(A=1|W=0, G=0) 
ordered_hidden_var_list = ["B","E"]
evidence_vars =[["G",0],["W",0]]
query_variables=["A"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Multiplying:
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799

In [ ]:
#p(A=1|W=0, G=0)=0.00679

In [55]:
#4A
#P(B = 1) ?= p(B=1|W=1) 
#P(B)
ordered_hidden_var_list = ["A","E","G","W"]
evidence_vars =[]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [75]:
#4A
#P(B) = 0.0001
#P(B = 1) ?= p(B=1|W=1)  
# = 0.000193
#NO
ordered_hidden_var_list = ["A","E","G"]
evidence_vars =[["W",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [57]:
#4A
#P(B = 1) ?= p(B=1|G=1) 
#p(B=1|G=1) = 0.000193
#P(B) = 0.0001
#NO
ordered_hidden_var_list = ["A","E","W"]
evidence_vars =[["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [156]:
#4A
#P(B = 1) ?= p(B=1|A=1) 
#p(B=1|A=1) = 0.009359
#P(B) = 0.0001
#NO
ordered_hidden_var_list = ["E","G","W"]
evidence_vars =[["A",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0  0.04
1  1.0  1.0  0.40
0  0.0

In [83]:
#4B 
#check
#P(B=1|W=1) ?= P(B=1|G=1,W=1)
#P(B=1|W=1) ?= P(B=1|A=1,W=1)
#P(B=1|W=1)  =  0.000193
ordered_hidden_var_list = ["A","E","G"]
evidence_vars =[["W",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [85]:
#4B part1
#check
#P(B=1|W=1) ?= P(B=1|G=1,W=1)
#P(B=1|W=1)  =  0.000193
# P(B=1|G=1,W=1) = 0.000374
ordered_hidden_var_list = ["A","E"]
evidence_vars =[["W",1],["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [86]:
#4B  -part2
#check
#P(B=1|W=1) ?= P(B=1|A=1,W=1)
#P(B=1|W=1)  =  0.000193
#P(B=1|A=1,W=1) = 0.009359
ordered_hidden_var_list = ["E","G"]
evidence_vars =[["A",1],["W",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [87]:
#4c
#check
#P(B=1|G=1) ?= P(B=1|G=1,W=1)
#P(B=1|G=1) ?= P(B=1|G=1,A=1)
#P(B=1|G=1)  =  0.000193
ordered_hidden_var_list = ["A","E","W"]
evidence_vars =[["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [88]:
#4c part1
#check
#P(B=1|G=1) ?= P(B=1|G=1,W=1)
#P(B=1|G=1)  =  0.000193
# P(B=1|G=1,W=1) = 0.000374
ordered_hidden_var_list = ["A","E"]
evidence_vars =[["W",1],["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [89]:
#4c  -part2
#check
#P(B=1|G=1) ?= P(B=1|A=1,G=1)
#P(B=1|G=1) = 0.000193
#P(B=1|G=1,A=1) = 0.009359
ordered_hidden_var_list = ["E","W"]
evidence_vars =[["A",1],["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [91]:
#4d
#check
#P(B=1|A=1) ?= P(B=1|A=1,W=1)
#P(B=1|A=1) ?= P(B=1|A=1,G=1)
#P(B=1|A=1)  = 0.009359
ordered_hidden_var_list = ["E","G","W"]
evidence_vars =[["A",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [94]:
#P(B=1|A=0)  = 0.000005
ordered_hidden_var_list = ["E","G","W"]
evidence_vars =[["A",0]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [90]:
#4d part1
#check
#P(B=1|A=1) ?= P(B=1|A=1,W=1)
#P(B=1|A=1)  = 0.009359
#P(B=1|A=1,W=1) = 0.009359
ordered_hidden_var_list = ["E","G"]
evidence_vars =[["A",1],["W",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [92]:
#4d  -part2
#check
#P(B=1|A=1) ?= P(B=1|A=1,G=1)
#P(B=1|A=1)  = 0.009359
#P(B=1|A=1,G=1) = 0.009359
ordered_hidden_var_list = ["E","W"]
evidence_vars =[["A",1],["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [ ]:
#checked
#P(B=1|A=1) = P(B=1|A=1,W=1)
#also check 
#P(B=1|A=1) ?= P(B=1|A=1,W=0)
#P(B=1|A=0) ?= P(B=1|A=0,W=0)
#P(B=1|A=0) ?= P(B=1|A=0,W=1)

#checked
#P(B=1|A=1) = P(B=1|A=1,G=1)
#also check
#P(B=1|A=1) = P(B=1|A=1,G=0)
#P(B=1|A=0) = P(B=1|A=0,G=1)
#P(B=1|A=0) = P(B=1|A=0,G=0)



In [93]:
#P(B=1|A=1) ?= P(B=1|A=1,W=0)
#P(B=1|A=1)  = 0.009359
#P(B=1|A=1,W=0) = 0.009359
ordered_hidden_var_list = ["E","G"]
evidence_vars =[["A",1],["W",0]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [95]:
#P(B=1|A=0)  = 0.000005
#P(B=1|A=0) ?= P(B=1|A=0,W=0)
#P(B=1|A=0,W=0) = 0.000005
ordered_hidden_var_list = ["E","G"]
evidence_vars =[["A",0],["W",0]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [96]:
#P(B=1|A=0)  = 0.000005
#P(B=1|A=0) ?= P(B=1|A=0,W=1)
#P(B=1|A=0,W=1) = 0.000005
ordered_hidden_var_list = ["E","G"]
evidence_vars =[["A",0],["W",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [97]:
#also check
#P(B=1|A=1) = P(B=1|A=1,G=0)
#P(B=1|A=1)  = 0.009359
#P(B=1|A=1,G=0) = 0.009359
ordered_hidden_var_list = ["E","W"]
evidence_vars =[["A",1],["G",0]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [98]:
#P(B=1|A=0) = P(B=1|A=0,G=1)
# P(B=1|A=0) = 0.000005
#P(B=1|A=0,G=1) = 0.000005
ordered_hidden_var_list = ["E","W"]
evidence_vars =[["A",0],["G",1]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [99]:
#P(B=1|A=0) = P(B=1|A=0,G=0)
# P(B=1|A=0) = 0.000005
#P(B=1|A=0,G=0) = 0.000005
ordered_hidden_var_list = ["E","W"]
evidence_vars =[["A",0],["G",0]]
query_variables=["B"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     E    prob
0  1.0  0.0003
0  0.0  0.9997
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
---------------
Sumout E from:
     E    A    B      prob
0  1.0  1.0  0.0  0.000060
1  1.0  1.0  1.0  0.000288
2  1.0  0.0  0.0  0.000240
3  1.0  0.0  1.0  0.000012
4  0.0  1.0  0.0  0.009997
5  0.0  1.0  1.0  0.949715
6  0.0  0.0  0.0  0.989703
7  0.0  0.0  1.0  0.049985
Result: 
     A    B      prob
0  0.0  0.0  0.989943
1  0.0  1.0  0.049997
2  1.0  0.0  0.010057
3  1.0  1.0  0.950003
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [ ]:
#5A 
#P(E=1|A=1) ? = P(E=1|A=1,W=1)
#P(E=1|A=1) ? = P(E=1|A=1,G=1)
#P(E=1|A=1) ? = P(E=1|A=1,WE=1)

In [100]:
#5A 
#P(E=1|A=1)  = 0.005913
ordered_hidden_var_list = ["B","G","W"]
evidence_vars =[["A",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [101]:
#5A 
#P(E=1|A=1)  = 0.005913
#P(E=1|A=1,W=1) = 0.005913
#then also need to check 
#P(E=1|A=1) ? = P(E=1|A=1,W=0)
#P(E=1|A=0) ? = P(E=1|A=0,W=1)
#P(E=1|A=0) ? = P(E=1|A=0,W=0)
ordered_hidden_var_list = ["B","G"]
evidence_vars =[["A",1],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [102]:
#5A 
#P(E=1|A=1)  = 0.005913
#then also need to check 
#P(E=1|A=1) = P(E=1|A=1,W=0) = 0.005913
ordered_hidden_var_list = ["B","G"]
evidence_vars =[["A",1],["W",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [103]:
#5A 
#P(E=1|A=0) = 0.000242
ordered_hidden_var_list = ["B","G","W"]
evidence_vars =[["A",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [135]:
#P(G|W) 
ordered_hidden_var_list = ["A","B","E"]
evidence_vars =[["W",0]]
query_variables=["G"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0  0.016
4   0.0  0.0  0.0  0.0  0.594
5   0.0  0.0  0.0  1.0  0.480
6   0.0  0.0  1.0  0.0  0.030
7   0.0  0.0  1.0  1.0  0.024
8   1.0  1.0  0.0  0.0  0.008
9   1.0  1.0  0.0  1.0  0.160
10  1.0  1.0  1.0  0.0  0.760
11  1.0  1.0  1.0  1.0  0.768
12  0.0  1.0  0.0  0.0  0.002
13  0.0  1.0  0.0  1.0  0.040
14  0.0  1.0  1.0  0.0  0.190
15  0.0  1.0  1.0  1.0  0.192
---------------
Multiplying:
      W    A    B    E   prob
0   1.0  0.0  0.0  0.0  0.396
1   1.0  0.0  0.0  1.0  0.320
2   1.0  0.0  1.0  0.0  0.020
3   1.0  0.0  1.0  1.0 

In [104]:
#5A 
#P(E=1|A=0) = 0.000242
#then also need to check 
#P(E=1|A=0)  = P(E=1|A=0,W=1) = 0.000242
ordered_hidden_var_list = ["B","G"]
evidence_vars =[["A",0],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [158]:
#5A 
#P(E=1|A=0)  = 0.000242
#P(E=1|A=0) = P(E=1|A=0,W=0) not equal 
ordered_hidden_var_list = ["B","G"]
evidence_vars =[["A",0],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout G from:
     G    A  prob
0  1.0  0.0  0.04
1  1.0  1.0  0.40
0  0.0

In [159]:
#5A- part2
#P(E=1|A=1) = 0.005913
#P(E=1|A=1) = P(E=1|A=1,G=1)  = 0.005913
#also need to check 
#P(E=1|A=1) = P(E=1|A=1,G=0)
#P(E=1|A=0) = P(E=1|A=0,G=1)
#P(E=1|A=0) = P(E=1|A=0,G=0)
ordered_hidden_var_list = ["B","W"]
evidence_vars =[["A",1],["G",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [160]:
#5A- part2
#P(E=1|A=1) = 0.005913
#also need to check 
#P(E=1|A=1) = P(E=1|A=1,G=0) = 0.005913
ordered_hidden_var_list = ["B","W"]
evidence_vars =[["A",1],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)


Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [161]:
#5A- part2
#P(E=1|A=0)  = 0.000242
#also need to check 
#P(E=1|A=0) = P(E=1|A=0,G=1) = 0.000242
ordered_hidden_var_list = ["B","W"]
evidence_vars =[["A",0],["G",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)


Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [109]:
#5A- part2
#P(E=1|A=0)  = 0.000242
#also need to check 
#P(E=1|A=0) = P(E=1|A=0,G=0) = 0.000242
ordered_hidden_var_list = ["B","W"]
evidence_vars =[["A",0],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)



Multiplying:
     B    prob
0  1.0  0.0001
0  0.0  0.9999
     A    B    E  prob
0  1.0  0.0  0.0  0.01
1  1.0  0.0  1.0  0.20
2  1.0  1.0  0.0  0.95
3  1.0  1.0  1.0  0.96
0  0.0  0.0  0.0  0.99
1  0.0  0.0  1.0  0.80
2  0.0  1.0  0.0  0.05
3  0.0  1.0  1.0  0.04
Result:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
---------------
Sumout B from:
     B    A    E      prob
0  1.0  1.0  0.0  0.000095
1  1.0  1.0  1.0  0.000096
2  1.0  0.0  0.0  0.000005
3  1.0  0.0  1.0  0.000004
4  0.0  1.0  0.0  0.009999
5  0.0  1.0  1.0  0.199980
6  0.0  0.0  0.0  0.989901
7  0.0  0.0  1.0  0.799920
Result: 
     A    E      prob
0  0.0  0.0  0.989906
1  0.0  1.0  0.799924
2  1.0  0.0  0.010094
3  1.0  1.0  0.200076
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0

In [162]:
#5A- part3
#P(E=1|A=1) = 0.005913
#also need to check 
#P(E=1|A=1) = P(E=1|A=1,B=1) = 0.000303
ordered_hidden_var_list = ["G","W"]
evidence_vars =[["A",1],["B",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0  0.04
1  1.0  1.0  0.40
0  0.0  0.0  0.96
1  0.0  1.0  0.60
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     A  prob
0  0.0   1.0
1  1.0   1.0
Result:
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-

In [111]:
#5B part1
#check p(E = 1|A = 1,B = 1) = p(E = 1|A = 1,B = 1, G = 1)
#p(E = 1|A = 1,B = 1) = 0.000303

ordered_hidden_var_list = ["G","W"]
evidence_vars =[["A",1],["B",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     A  prob
0  0.0   1.0
1  1.0   1.0
Result:
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-

In [163]:
#5B part1
#check p(E = 1|A = 1,B = 1) = p(E = 1|A = 1,B = 1, G = 1)
#p(E = 1|A = 1,B = 1) = 0.000303
# p(E = 1|A = 1,B = 1, G = 1) = 0.000303
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",1],["B",1],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [ ]:
#5B part1
#check p(E = 1|A = 1,B = 1) = p(E = 1|A = 1,B = 1, G = 1)
#p(E = 1|A = 1,B = 1) = 0.000303
#also check 
#p(E = 1|A = 1,B = 1) = p(E = 1|A = 1,B = 1, G = 0)

#p(E = 1|A = 1,B = 0) = p(E = 1|A = 1,B = 0, G = 0)
#p(E = 1|A = 1,B = 0) = p(E = 1|A = 1,B = 0, G = 1)

#p(E = 1|A = 0,B = 1) = p(E = 1|A = 0,B = 1, G = 0)
#p(E = 1|A = 0,B = 1) = p(E = 1|A = 0,B = 1, G = 1)


#p(E = 1|A = 0,B = 0) = p(E = 1|A = 0,B = 0, G = 0)
#p(E = 1|A = 0,B = 0) = p(E = 1|A = 0,B = 0, G = 1)




In [ ]:
#p(E = 1|A = 1,B = 1) = 0.000303
#also need to calculate 
#p(E = 1|A = 0,B = 0) = 0.000242
#p(E = 1|A = 1,B = 0) = 0.005966
#p(E = 1|A = 0,B = 1) = 0.00024

In [116]:
#ordered_hidden_var_list = ["G","W"]
#evidence_vars =[["A",0],["B",1]]
#query_variables=["E"]
#inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

In [164]:
#5B part1
#check p(E = 1|A = 1,B = 1) = p(E = 1|A = 1,B = 1, G = 1)
#p(E = 1|A = 1,B = 1) = 0.000303
#also check 
#p(E = 1|A = 1,B = 1) = p(E = 1|A = 1,B = 1, G = 0) = 0.000303
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",1],["B",1],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [118]:
#p(E = 1|A = 1,B = 0) = p(E = 1|A = 1,B = 0, G = 0) = 0.005966
#p(E = 1|A = 1,B = 0) = 0.005966
#p(E = 1|A = 1,B = 0, G = 0) = 0.005966
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",1],["B",0],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)


Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [119]:
#p(E = 1|A = 1,B = 0) = 0.005966
#p(E = 1|A = 1,B = 0) = p(E = 1|A = 1,B = 0, G = 1) = 0.005966
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",1],["B",0],["G",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [120]:
#p(E = 1|A = 0,B = 1) = p(E = 1|A = 0,B = 1, G = 0)
#p(E = 1|A = 0,B = 1) = 0.00024
#p(E = 1|A = 0,B = 1, G = 0) = 0.00024
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",0],["B",1],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [121]:
#p(E = 1|A = 0,B = 1) = p(E = 1|A = 0,B = 1, G = 1)
#p(E = 1|A = 0,B = 1) = 0.00024
#p(E = 1|A = 0,B = 1, G = 1) = 0.00024
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",0],["B",1],["G",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [122]:
#p(E = 1|A = 0,B = 0) = p(E = 1|A = 0,B = 0, G = 0)
#p(E = 1|A = 0,B = 0)  = 0.000242
#p(E = 1|A = 0,B = 0, G = 0)
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",0],["B",0],["G",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [124]:
#p(E = 1|A = 0,B = 0) = p(E = 1|A = 0,B = 0, G = 1)
#p(E = 1|A = 0,B = 0)  = 0.000242
#p(E = 1|A = 0,B = 0, G = 1) = 0.000242
ordered_hidden_var_list = ["W"]
evidence_vars =[["A",0],["B",0],["G",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout W from:
     W    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [125]:
#5B 
#check p(E=1|A=1,B=1) ?=  p(E=1|A=1,B=1,W=1) = 0.000303
#p(E = 1|A = 1,B = 1) = 0.000303
#p(E=1|A=1,B=1,W=1) = 0.000303
#also need check 
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",1],["B",1],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [ ]:
#5B 
#also need to check 
#p(E = 1|A = 1,B = 1) = 0.000303
#also need to calculate 
#p(E = 1|A = 0,B = 0) = 0.000242
#p(E = 1|A = 1,B = 0) = 0.005966
#p(E = 1|A = 0,B = 1) = 0.00024

In [ ]:
#5B 
#check p(E=1|A=1,B=1) ?=  p(E=1|A=1,B=1,W=1) = 0.000303
#p(E = 1|A = 1,B = 1) = 0.000303
#p(E=1|A=1,B=1,W=1) = 0.000303
#also need check 
#p(E=1|A=1,B=1) ?=  p(E=1|A=1,B=1,W=0) 


#p(E=1|A=1,B=0) ?=  p(E=1|A=1,B=0,W=1) 
#p(E=1|A=1,B=0) ?=  p(E=1|A=1,B=0,W=0) 

#p(E=1|A=0,B=1) ?=  p(E=1|A=0,B=1,W=1) 
#p(E=1|A=0,B=1) ?=  p(E=1|A=0,B=1,W=0) 


#p(E=1|A=0,B=0) ?=  p(E=1|A=0,B=0,W=1) 
#p(E=1|A=0,B=0) ?=  p(E=1|A=0,B=0,W=0) 



In [126]:
#5B 
#also need to check 
#p(E = 1|A = 1,B = 1) = 0.000303
#also need check 
#p(E=1|A=1,B=1) ?=  p(E=1|A=1,B=1,W=0)  = 0.000303
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",1],["B",1],["W",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [128]:
#p(E=1|A=1,B=0) ?=  p(E=1|A=1,B=0,W=1)  = 0.005966
#p(E = 1|A = 1,B = 0) = 0.005966
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",1],["B",0],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [129]:
#p(E=1|A=1,B=0) ?=  p(E=1|A=1,B=0,W=0)  = 0.005966
#p(E = 1|A = 1,B = 0) = 0.005966
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",1],["B",0],["W",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [130]:
#p(E=1|A=0,B=1) ?=  p(E=1|A=0,B=1,W=1)  = 0.00024
#p(E = 1|A = 0,B = 1) = 0.00024
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",0],["B",1],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [131]:
#p(E=1|A=0,B=1) ?=  p(E=1|A=0,B=1,W=0)  = 0.00024
#p(E = 1|A = 0,B = 1) = 0.00024
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",0],["B",1],["W",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [132]:
#p(E=1|A=0,B=0) ?=  p(E=1|A=0,B=0,W=1) = 0.000242
#p(E = 1|A = 0,B = 0) = 0.000242
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",0],["B",0],["W",1]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [133]:
#p(E=1|A=0,B=0) ?=  p(E=1|A=0,B=0,W=0) = 0.000242
#p(E = 1|A = 0,B = 0) = 0.000242
ordered_hidden_var_list = ["G"]
evidence_vars =[["A",0],["B",0],["W",0]]
query_variables=["E"]
inference(factor_list, query_variables, ordered_hidden_var_list, evidence_vars)

Sumout G from:
     G    A  prob
0  1.0  0.0   0.4
1  1.0  1.0   0.8
0  0.0  0.0   0.6
1  0.0  1.0   0.2
Result: 
     A  prob
0  0.0   1.0
1  1.0   1.0
---------------
Multiplying:
     A  prob
0  0.0   1.0
1  1.0   1.0
     E    prob
0  1.0  0.0003
0  0.0  0.9997
Result:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
---------------
Multiplying:
     A    E    prob
0  0.0  1.0  0.0003
1  0.0  0.0  0.9997
2  1.0  1.0  0.0003
3  1.0  0.0  0.9997
     B    prob
0  1.0  0.0001
0  0.0  0.9999
Result:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0  0.0  9.996000e-01
4  1.0  1.0  1.0  3.000000e-08
5  1.0  1.0  0.0  2.999700e-04
6  1.0  0.0  1.0  9.997000e-05
7  1.0  0.0  0.0  9.996000e-01
---------------
Multiplying:
     A    E    B          prob
0  0.0  1.0  1.0  3.000000e-08
1  0.0  1.0  0.0  2.999700e-04
2  0.0  0.0  1.0  9.997000e-05
3  0.0  0.0 

In [ ]:
#also check
#P(B=1|A=0) = 0.000005 (next cell)
# run in this order
#P(B=1|A=0)?=p(B=1|A=0,W=1,G=1) = 0.000005
#P(B=1|A=0)?=p(B=1|A=0,W=0,G=1) = 0.000005
#P(B=1|A=0)?=p(B=1|A=0,W=0,G=0) = 0.000005
#P(B=1|A=0)?=p(B=1|A=0,W=1,G=1) = 0.000005

#and 
#P(B=1|A=1)  = 0.009359 (from pervious cell)
#P(B=1|A=1)?=p(B=1|A=1,W=1,G=1) = 0.009359
#P(B=1|A=1)?=p(B=1|A=1,W=0,G=1) = 0.009359
#P(B=1|A=1)?=p(B=1|A=1,W=0,G=0) = 0.009359 
#P(B=1|A=1)?=p(B=1|A=1,W=1,G=1) = 0.009359